# Агент-риелтор на GigaChat functions
В этом примере, мы создадим агента по продаже недвижимости, который в формате диалога может:
* искать квартиры по нашей внутренней "базе-данных"
* бронировать понравившиеся пользователю квартиры
* рассчитывать по ним ипотеку
* уточнять недостающую информацию у пользователя
* хранить весь диалог и информацию, которую мы получили от пользователя и результаты выполнения функций

In [8]:
%pip install --upgrade --quiet gigachain


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
import csv
from typing import Type, Optional

from langchain.agents import create_gigachat_functions_agent
from langchain.agents.gigachat_functions_agent.base import (
    format_to_gigachat_function_messages,
)
from langchain_community.chat_models import GigaChat
from langchain.tools import BaseTool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.pydantic_v1 import BaseModel, Field
from langchain.agents import (
    AgentExecutor,
)

In [2]:
import os
from getpass import getpass

os.environ["GIGACHAT_CREDENTIALS"] = getpass()
os.environ["GIGACHAT_SCOPE"] = getpass()

 ········
 ········


Создаем нашу "базу-данных" из тестового csv файла

In [25]:
realestate_database = {}
with open("flats.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row["комнат"] = int(row["комнат"])
        row["площадь"] = int(row["площадь"])
        row["Цена"] = int(row["Цена"]) * 1000
        row["отделка"] = "есть" if row["отделка"] == "TRUE" else "нет"
        realestate_database[row["id"]] = row

Подгружаем описание о ЖК

In [26]:
with open("description.txt", "r", encoding="utf-8") as file:
    description = file.read()

Создаем промпт и инициализируем llm

In [27]:
system = f"""Ты агент по продаже недвижимости в ЖК Life Варшавская.
У тебя есть доступ к базе данных и ты должен помочь пользователю выбрать квартиру под его запросы.
Говори только то, что было сообщено тебе в данных и результатах поиска.
Ты должен помочь пользователю выбрать квартиру и оформить заказ на неё.
Если клиент заинтересовался покупкой, то ты должен забронировать за ним квартиру с помощью функции book_flat.

Также у тебя есть доступные функции:
Для бронирования квартиры используй book_flat
Для поиска доступных квартир используй search
Для связи с менеджером call_manager
Для расчета ипотеки по квартире loan_calculator

Перед бронированием узнай у человека его имя и телефон, не пытайся их придумать

После бронирования квартиры предложи пользователю рассчитать ипотеку. Если он согласится и передаст тебе нужные данные, то выполни расчет.

Не пиши одно и тоже пользователю.
Бери данные только из диалога, когда пользователь явно сообщил тебе их. Не придумывай данные сам.
Если каких-то данных не хватает для вызова функции, то нужно спросить данные у пользователя.

Вот данные по ЖК: {description}
"""  # noqa

In [28]:
llm = GigaChat(
    verify_ssl_certs=False,
    timeout=600,
    model="GigaChat-Pro-preview",
)

## Блок создания tool'ов
### Tool для поиска квартир

In [39]:
class SearchInput(BaseModel):
    min_area: int = Field(
        description="минимальная площадь квартиры в метрах", default=0
    )
    max_area: int = Field(
        description="максимальная площадь квартиры",
        default=150,
    )
    price_min: Optional[int] = Field(
        description="минимальная цена квартиры", default=None
    )
    price_max: Optional[int] = Field(
        description="максимальная цена квартиры", default=None
    )
    rooms_min: int = Field(
        description="минимальное количество комнат. Если 0, то это квартира-студия",
        default=0,
    )
    rooms_max: Optional[int] = Field(
        description="максимальное количество комнат", default=5
    )


class SearchTool(BaseTool):
    name = "search"
    description = """Выполняет поиск квартиры в базе данных по параметрам.

Нужно найти только двухкомнатные квартиры, то укажи {rooms_min: 2, rooms_max: 2}

Нужно найти только студии, то укажи {rooms_min: 0, rooms_max: 0}

Нужно найти только квартиры дешевле 20 миллионов, то укажи {price_max: 20000000}

Нужно найти только квартиры дороже 10 миллионов, то укажи {price_min: 10000000}
"""
    args_schema: Type[BaseModel] = SearchInput

    def _run(
        self,
        min_area: int = 0,
        max_area: int = 150,
        price_min: Optional[int] = None,
        price_max: Optional[int] = None,
        rooms_min: int = 0,
        rooms_max: int = 5,
        run_manager=None,
    ) -> str:
        print(
            "\033[92m" + f"Searching min_area: {min_area}, max_area: {max_area}, "
            f"price_min: {price_min}, price_max: {price_max}, "
            f"rooms_min: {rooms_min}, rooms_max: {rooms_max}" + "\033[0m"
        )
        price_min = 0 if price_min is None else price_min
        price_max = 100 * 1000 * 1000 if price_max is None else price_max
        result = []
        for flat in realestate_database.values():
            if (
                min_area <= flat["площадь"] <= max_area
                and price_min <= flat["Цена"] <= price_max
                and rooms_min <= flat["комнат"] <= rooms_max
            ):
                result.append(flat)

        if len(result) == 0:
            return "Ничего не найдено"
        else:
            result_string = "Найденные квартиры:\n"
            for item in result[:5]:
                result_string += "\n".join(
                    [f"{key}: {value}" for key, value in item.items()]
                )
                result_string += "\n-----\n"
            return (
                result_string
                + "Для бронирования квартиры используй book_flat"
            )

### Tool для бронирования квартиры

In [30]:
class BookInput(BaseModel):
    flat_id: Optional[str] = Field(description="id квартиры")
    phone: str = Field(
        description="телефон пользователя, который выполняет бронирование"
    )
    name: str = Field(description="имя пользователя, который выполняет бронирование")
    additional_info: str = Field(
        description="дополнительная информация о бронировании", default=""
    )


class BookTool(BaseTool):
    name = "book_flat"
    description = """Бронирует квартиру.
Возвращает сообщение о результате бронирования.
Перед тем как вызвать функцию узнай имя пользователя и его телефон.
Не выдумывай их

Примеры:
Нужно забронировать квартиру (пользователь выбрал квартиру) {flat_id: id квартиры, name: имя, phone: телефон}

"""  # noqa
    args_schema: Type[BaseModel] = BookInput

    def _run(
        self,
        phone: str,
        name: str,
        flat_id: str = None,
        additional_info: str = "",
        run_manager=None,
    ) -> str:
        print(
            "\033[92m"
            + f"!!! Booking flat {flat_id} for {phone} {name} {additional_info}"
            + "\033[0m"
        )
        if flat_id not in realestate_database:
            return (
                "Не получилось забронировать квартиру. "
                "Выясни ID квартиры с помощью search"
            )
        if not name or not phone:
            return "Узнай имя и телефон пользователя. Потом вызови эту функцию снова"
        return (
            f"Квартира забронирована."
            f"Спроси пользователя хочет ли он рассчитать ипотеку по этой квартире"
        )

### Tool для расчета ипотеки

In [31]:
class CalculatorInput(BaseModel):
    price: Optional[str] = Field(description="цена квартиры", default=None)
    years: Optional[int] = Field(
        description="количество лет на которое берется ипотека", default=None
    )
    first_pay_percent: int = Field(
        description="процент первоначального взноса", default=20
    )


class CalculatorTool(BaseTool):
    name = "loan_calculator"
    description = (
        """Выполняет расчет стоимости забронированной квартиры в ипотеку."""  # noqa
    )
    args_schema: Type[BaseModel] = CalculatorInput

    def _run(
        self,
        price: int,
        years: Optional[int] = None,
        first_pay_percent: int = 20,
        run_manager=None,
    ) -> str:
        print(
            "\033[92m"
            + f"loan_calculator years: {years}, price: {price}, first_pay_percent: {first_pay_percent}"
            + "\033[0m"
        )
        if not years:
            return "Уточни на какой срок ипотека. И вызови эту функцию снова"
        price = int(price)
        loan_amount = price - price * first_pay_percent / 100
        monthly_payment = round(loan_amount * (1 + 0.1) / (years * 12))
        res = (
            f"Сумма кредита: {loan_amount} рублей, "
            f"ежемесячный платеж: {monthly_payment} рублей, "
            f"срок кредита: {years}. Расскажи пользователю информацию по ипотеке"
        )
        return res

### Tool для вызова менеджера

In [32]:
class CallManagerInput(BaseModel):
    phone: str = Field(
        description="телефон пользователя, который выполняет бронирование"
    )
    name: str = Field(description="имя пользователя, который выполняет бронирование")
    additional_info: str = Field(
        description="дополнительная информация о бронировании", default=""
    )


class CallManagerTool(BaseTool):
    name = "call_manager"
    description = """Связывает пользователя с менеджером.
Перед тем как вызвать функцию нужно узнать имя клиента и телефон."""  # noqa
    args_schema: Type[BaseModel] = CallManagerInput

    def _run(
        self,
        phone: str,
        name: str,
        additional_info: str = "",
        run_manager=None,
    ) -> str:
        print(
            "\033[92m"
            + f"!!! call_manager for {phone} {name} {additional_info}"
            + "\033[0m"
        )
        if not name or not phone:
            return (
                "Узнай имя и телефон пользователя и вызови функцию call_manager снова"
            )
        return "Менеджер свяжется в течении 5 минут"

## Создаем агента

In [40]:
tools = [BookTool(), SearchTool(), CalculatorTool(), CallManagerTool()]
agent = create_gigachat_functions_agent(llm, tools)

# AgentExecutor создает среду, в которой будет работать агент
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=False, return_intermediate_steps=True
)

## Примеры
### Поиск квартиры, бронирование и расчет ипотеки

In [35]:
import time

chat_history = [SystemMessage(content=system)]
while True:
    user_input = input("Покупатель: ")
    print(f"Покупатель: {user_input}")
    if user_input == "":
        break
    result = agent_executor.invoke(
        {
            "chat_history": chat_history,
            "input": user_input,
        }
    )
    print("\033[93m" + f"Bot: {result['output']}" + "\033[0m")
    chat_history.append(HumanMessage(content=user_input))
    chat_history += format_to_gigachat_function_messages(result["intermediate_steps"])
    chat_history.append(AIMessage(content=result["output"]))
    time.sleep(0.3)

Покупатель: Какие есть квартиры дешевле 30 миллионов
Searching min_area: 0, max_area: 150, price_min: None, price_max: 30000000, rooms_min: 0, rooms_max: 5
Bot: Есть несколько вариантов квартир дешевле 30 миллионов рублей. Первая - студия площадью 26 квадратных метров на 13 этаже корпуса К6-1 за 11 502 000 рублей. Вторая - двухкомнатная квартира площадью 73 квадратных метра на 8 этаже того же корпуса за 26 183 000 рублей.
Покупатель: забронируй двухкомнатную
!!! Booking flat 2 for   
Bot: Конечно, я могу забронировать эту квартиру за вами. Но сначала мне нужно узнать ваше имя и номер телефона. Это необходимо для создания брони.
Покупатель: Михаил 88005553535
!!! Booking flat 2 for 88005553535 Михаил 
Bot: Хотели бы вы рассчитать ипотеку по этой квартире?
Покупатель: Да
Bot: Отлично! Я могу помочь вам с этим. Не могли бы Вы предоставить мне общую стоимость квартиры и желаемый срок кредита в годах?
Покупатель: Рассчитай ипотеку по забронированной мной квартире
loan_calculator years: 20, 

### Последовательное выполнение функций
Предположим, что у нас пользователь не хочет искать квартиры, а хочет сразу забронировать первую попавшуюся :)
Получается наш агент сначала попытается забронировать квартиру, которую не знает, а значит пойдет искать с поиском,
найдет и после этого забронирует квартиру.
Примерная логика такая:
1. пользователь просит забронировать квартиру студию
2. бот не знает id квартиры
3. вызывает функцию бронирования, которая говорит, что нужно сначала воспользоваться поиском
4. бот ищет доступные квартиры
5. снова вызывает функцию бронирования исходя из инфы поиска

In [41]:
import time

chat_history = [SystemMessage(content=system)]
while True:
    user_input = input("Покупатель: ")
    print(f"Покупатель: {user_input}")
    if user_input == "":
        break
    result = agent_executor.invoke(
        {
            "chat_history": chat_history,
            "input": user_input,
        }
    )
    print("\033[93m" + f"Bot: {result['output']}" + "\033[0m")
    chat_history.append(HumanMessage(content=user_input))
    chat_history += format_to_gigachat_function_messages(result["intermediate_steps"])
    chat_history.append(AIMessage(content=result["output"]))
    time.sleep(0.3)

Покупатель: Забронируй мне студию
Bot: Конечно, я могу помочь с этим. Не могли бы Вы предоставить мне Ваше имя и номер телефона?
Покупатель: Михаил 88005553535
!!! Booking flat None for 88005553535 Михаил 
Searching min_area: 0, max_area: 150, price_min: None, price_max: None, rooms_min: 0, rooms_max: 0
!!! Booking flat 1 for 88005553535 Михаил 
Bot: Хотели бы Вы рассчитать ипотеку по данной квартире?
Покупатель: Давай
loan_calculator years: None, price: 11502000, first_pay_percent: 20
Bot: На какой срок ипотеки Вы хотели бы рассчитать ежемесячный платеж?
Покупатель: 5 лет
loan_calculator years: 5, price: 11502000, first_pay_percent: 20
Bot: Исходя из суммы кредита и срока кредита, Ваш ежемесячный платеж по ипотеке будет примерно 168696 рублей.
Покупатель: 
